In [1]:
import pandas as pd
import numpy as np

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

**Предобработка данных**

In [2]:
df = pd.read_csv('regulations.csv', index_col='id')
df.head()

,act_title,publication_date,developer,okved_list,views_num,comments_num,likes_num,dislikes_num,regulatory_impact,added_by,responsible,is_regionally_signigicant,act_changes_controlling_activities,mineco_solution,problem_addressed,act_objectives,persons_affected_by_act,relations_regulated_by_act,act_significance
id,,,,,,,,,,,,,,,,,,,
5038,Об утверждении тарифов на услуги по транспорти...,2013-09-11,ФСТ России,NaN,376.0,0.0,0.0,0.0,Низкая,Митина Ольга Викторовна,Митина Ольга Викторовна,False,False,Не определено,NaN,NaN,NaN,NaN,NaN
5039,О внесении изменений в отдельные законодательн...,2013-06-11,Минтруд России,Здравоохранение; Предоставление социальных услуг,504.0,0.0,0.0,0.0,Низкая,Рахов Виталий Сергеевич,Павлова Зоя Ивановна,False,False,Не определено,NaN,NaN,NaN,NaN,NaN
5040,Об утверждении Положения об уведомлении лиц об...,2013-04-29,Росфинмониторинг,Финансовая деятельность,428.0,0.0,0.0,0.0,Низкая,Тимофеева Алёна Игоревна,Лях Валерий Владимирович,False,False,Не определено,NaN,NaN,NaN,NaN,NaN
5041,О внесении изменений в Положение о Министерств...,2013-10-21,Минобрнауки России,Образование,376.0,0.0,0.0,0.0,Низкая,Вотоновская Ирина Вячеславовна,Михайлова Ирина Вячеславовна,False,False,Не определено,NaN,NaN,NaN,NaN,NaN
5042,О внесении изменений в Правила подготовки и пр...,2014-02-24,Минприроды России,NaN,499.0,0.0,0.0,0.0,Низкая,Соболева Светлана Юрьевна,Соболева Светлана Юрьевна,False,False,Не определено,предоставление водного объекта в пользование п...,Пунктом 12 части 2 статьи 11 Водного кодекса Р...,неопределенный круг лиц,необходимость корреляции Правил подготовки и п...,Проект постановления Правительства Российской ...


Выделим столбцы данных, не нуждающихся в специфической предобработке.

In [3]:
new_df = df[['mineco_solution', 'added_by', 'responsible', 'developer', 
            'is_regionally_signigicant', 'act_changes_controlling_activities',
            'views_num', 'comments_num', 'likes_num', 'dislikes_num']]
new_df.head()

,mineco_solution,added_by,responsible,developer,is_regionally_signigicant,act_changes_controlling_activities,views_num,comments_num,likes_num,dislikes_num
id,,,,,,,,,,
5038,Не определено,Митина Ольга Викторовна,Митина Ольга Викторовна,ФСТ России,False,False,376.0,0.0,0.0,0.0
5039,Не определено,Рахов Виталий Сергеевич,Павлова Зоя Ивановна,Минтруд России,False,False,504.0,0.0,0.0,0.0
5040,Не определено,Тимофеева Алёна Игоревна,Лях Валерий Владимирович,Росфинмониторинг,False,False,428.0,0.0,0.0,0.0
5041,Не определено,Вотоновская Ирина Вячеславовна,Михайлова Ирина Вячеславовна,Минобрнауки России,False,False,376.0,0.0,0.0,0.0
5042,Не определено,Соболева Светлана Юрьевна,Соболева Светлана Юрьевна,Минприроды России,False,False,499.0,0.0,0.0,0.0


Избавимся от пропусков в имеющихся данных

In [4]:
for column_name in ['mineco_solution', 'added_by', 'responsible', 'developer', 
                    'is_regionally_signigicant', 'act_changes_controlling_activities']:
    new_df.loc[:, column_name] = new_df[column_name].copy().fillna('?').astype('category')

In [5]:
for column_name in ['views_num', 'comments_num', 'likes_num', 'dislikes_num']:
    new_df['no_' + column_name] = new_df[column_name].isna().astype('category')
    new_df.loc[:, column_name] = new_df[column_name].fillna(method='bfill')

Введём признак, обозначающий, что одно и то же лицо является одновременно создателем и ответственным

In [6]:
new_df['same_person'] = new_df[['added_by', 'responsible']].apply(lambda x: x['added_by'] == x['responsible'], 
                                                                  axis=1).astype('category')

Преобразуем столбец `publication_date` в числовые столбцы <<месяц>> и <<день>> (<<год>> не нужен, так как множества возможных годов у тренировочной и тетовой выборок не совпадают) и категориальный <<дата публикации отсутствует>>.

In [7]:
new_df['no_date'] = df['publication_date'].isna().astype('category')
new_df['month'] = df['publication_date'].fillna(method='bfill').apply(lambda x: float(x[5:7]))
new_df['day'] = df['publication_date'].fillna(method='bfill').apply(lambda x: float(x[8:]))

Столбец `regulatory_impact` можно разделить на 2 признака: порядковый (сам `regulatory_impact`) и категориальный (определен ли он). 

In [8]:
def impact(x):
    if x == 'Низкая':
        return 0.0
    if x == 'Средняя':
        return 1.0
    if x == 'Высокая':
        return 2.0
    return np.random.choice([0.0, 1.0, 2.0])

In [9]:
new_df.loc[:, 'regulatory_impact'] = df['regulatory_impact'].apply(impact)
new_df['impact_knowledge'] = df['regulatory_impact'].apply(lambda x: '?' if type(x) != str 
                                                         else '-' if x == 'Не определена' else '+').astype('category')

В столбце `okved_list` содержатся списки затронутых законопроектом сфер экономической деятельности. Его можно преобразовать в One Hot Encoding.

In [10]:
a = df['okved_list'].apply(lambda x: str(x).split(';'))
sa = set(map(lambda x: x.strip(), sum(a, [])))
sa.remove('nan')
print(sa)

{'Оптовая и розничная торговля', 'Текстильное и швейное производство', 'Пластмассовые изделия', 'Химическое производство', 'Здравоохранение', 'Производство электрооборудования', 'Производство электронного оборудования', 'Транспорт', 'Целлюлозно-бумажное производство', 'Образование', 'Добыча топливно-энергетических полезных ископаемых', 'Правоохранительная деятельность', 'Обеспечение военной безопасности', 'Производство бытовых изделий и предметов личного пользования', 'Производство резиновых изделий', 'Обработка древесины и производство изделий из дерева', 'Налоговое администрирование', 'Финансовая деятельность', 'Предоставление социальных услуг', 'Регистрация права собственности и постановка на кадастровый учет', 'Ремонт автотранспортных средств, мотоциклов', 'Гостиницы и рестораны', 'Издательская и полиграфическая деятельность', 'Производство прочих неметаллических минеральных продуктов', 'Производство кокса', 'Персональные данные', 'Государственное управление', 'Производство транспо

In [11]:
for cls in tqdm(sa):
    new_df.loc[:, cls] = a.apply(lambda x: cls in x).astype('category')

100%|███████████████████████████████████████████| 57/57 [00:00<00:00, 92.48it/s]


In [12]:
new_df.head()

,mineco_solution,added_by,responsible,developer,is_regionally_signigicant,act_changes_controlling_activities,views_num,comments_num,likes_num,dislikes_num,...,Информационные технологии,Корпоративное управление,"Рыболовство, рыбоводство","Производство пищевых продуктов, включая напитки, и табака",Лесное хозяйство,Производство оптического оборудования,Ядерные материалы,Прочие производства,Строительство,Внешнеэкономическая деятельность
id,,,,,,,,,,,,,,,,,,,,,
5038,Не определено,Митина Ольга Викторовна,Митина Ольга Викторовна,ФСТ России,False,False,376.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
5039,Не определено,Рахов Виталий Сергеевич,Павлова Зоя Ивановна,Минтруд России,False,False,504.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
5040,Не определено,Тимофеева Алёна Игоревна,Лях Валерий Владимирович,Росфинмониторинг,False,False,428.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
5041,Не определено,Вотоновская Ирина Вячеславовна,Михайлова Ирина Вячеславовна,Минобрнауки России,False,False,376.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
5042,Не определено,Соболева Светлана Юрьевна,Соболева Светлана Юрьевна,Минприроды России,False,False,499.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


По текстовым столбцам строим признаки, основанные на вхождении ключевых слов.

In [13]:
def act_type(x):
    if type(x) != type(' '):
        x = ' '
    x = x.lower()
    if x.find('закон') >= 0:
        return 'закон'
    if x.find('указ') >= 0:
        return 'указ'
    if x.find('постановлени') >= 0:
        return 'постановление'
    if x.find('распоряжени') >= 0:
        return 'распоряжение'
    if x.find('приказ') >= 0:
        return 'приказ'
    return 'другое'

In [14]:
new_df['act_type'] = df['act_title'].apply(act_type).astype(object)
new_df['changes'] = df['act_title'].fillna('').apply(lambda x: x.lower().find('изменени') >= 0 
                                                     or x.lower().find('дополнени') >= 0).astype('category')
new_df['rules'] = df['act_title'].fillna('').apply(lambda x: x.lower().find('порядк') >= 0 
                                                   or x.lower().find('правил') >= 0
                                                   or x.lower().find('регламент') >= 0).astype('category')
new_df['repeal'] = df['act_title'].fillna('').apply(lambda x: x.lower().find('отмен') >= 0 
                                                   or x.lower().find('прекращ') >= 0
                                                   or x.lower().find('признан') >= 0).astype('category')
new_df['approval'] = df['act_title'].fillna('').apply(lambda x: x.lower().find('утвержден') >= 0).astype('category')
new_df['definition'] = df['act_title'].fillna('').apply(lambda x: x.lower().find('установлен') >= 0 
                                                        or x.lower().find('определен') >= 0).astype('category')

In [15]:
new_df['affects_foreigners'] = df['persons_affected_by_act'].fillna('').apply(lambda x: x.lower().find('иностран') >= 0).astype('category')
new_df['affects_organizations'] = df['persons_affected_by_act'].fillna('').apply(lambda x: x.lower().find('организац') >= 0
                                                                                 or x.lower().find('юридическ') >= 0).astype('category')
new_df['affects_individual_enterpreneurs'] = df['persons_affected_by_act'].fillna('').apply(lambda x: x.lower().find('индивидуал')
                                                                                            >= 0).astype('category')
new_df['affects_natural_persons'] = df['persons_affected_by_act'].fillna('').apply(lambda x: x.lower().find('физическ')
                                                                                   >= 0).astype('category')
new_df['affects_governmental_agencies'] = df['persons_affected_by_act'].fillna('').apply(lambda x: x.lower().find('государств') >= 0 
                                                                                         or x.lower().find('федерал') >= 0
                                                                                         or x.lower().find('власт') >= 0).astype('category')

In [16]:
new_df['is_protecting'] = df['relations_regulated_by_act'].fillna('').apply(lambda x: x.lower().find('защит') >= 0 
                                                                            or x.lower().find('сохран') 
                                                                            >= 0).astype('category')
new_df['is_organising'] = df['relations_regulated_by_act'].fillna('').apply(lambda x: x.lower().find('организац') >= 0 
                                                                            or x.lower().find('управ') 
                                                                            >= 0).astype('category')

In [17]:
new_df['objectives_reference_закон'] = df['act_objectives'].fillna('').apply(lambda x: x.lower().find('закон')
                                                                             >= 0).astype('category')
new_df['objectives_reference_указ'] = df['act_objectives'].fillna('').apply(lambda x: x.lower().find('указ') 
                                                                            >= 0).astype('category')
new_df['objectives_reference_постановление'] = df['act_objectives'].fillna('').apply(lambda x: x.lower().find('постановление') 
                                                                                     >= 0).astype('category')
new_df['objectives_reference_распоряжение'] = df['act_objectives'].fillna('').apply(lambda x: x.lower().find('распоряжение')
                                                                                    >= 0).astype('category')
new_df['objectives_reference_приказ'] = df['act_objectives'].fillna('').apply(lambda x: x.lower().find('приказ') 
                                                                              >= 0).astype('category')

In [18]:
new_df['significance_reference_закон'] = df['act_significance'].fillna('').apply(lambda x: x.lower().find('закон') 
                                                                                 >= 0).astype('category')
new_df['significance_reference_указ'] = df['act_significance'].fillna('').apply(lambda x: x.lower().find('указ') 
                                                                                >= 0).astype('category')
new_df['significance_reference_постановление'] = df['act_significance'].fillna('').apply(lambda x: x.lower().find('постановление') 
                                                                                         >= 0).astype('category')
new_df['significance_reference_распоряжение'] = df['act_significance'].fillna('').apply(lambda x: x.lower().find('распоряжение') 
                                                                                        >= 0).astype('category')
new_df['significance_reference_приказ'] = df['act_significance'].fillna('').apply(lambda x: x.lower().find('приказ') 
                                                                                  >= 0).astype('category')

Вид данных после предобработки:

In [19]:
new_df.head()

,mineco_solution,added_by,responsible,developer,is_regionally_signigicant,act_changes_controlling_activities,views_num,comments_num,likes_num,dislikes_num,...,objectives_reference_закон,objectives_reference_указ,objectives_reference_постановление,objectives_reference_распоряжение,objectives_reference_приказ,significance_reference_закон,significance_reference_указ,significance_reference_постановление,significance_reference_распоряжение,significance_reference_приказ
id,,,,,,,,,,,,,,,,,,,,,
5038,Не определено,Митина Ольга Викторовна,Митина Ольга Викторовна,ФСТ России,False,False,376.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
5039,Не определено,Рахов Виталий Сергеевич,Павлова Зоя Ивановна,Минтруд России,False,False,504.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
5040,Не определено,Тимофеева Алёна Игоревна,Лях Валерий Владимирович,Росфинмониторинг,False,False,428.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
5041,Не определено,Вотоновская Ирина Вячеславовна,Михайлова Ирина Вячеславовна,Минобрнауки России,False,False,376.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
5042,Не определено,Соболева Светлана Юрьевна,Соболева Светлана Юрьевна,Минприроды России,False,False,499.0,0.0,0.0,0.0,...,False,False,True,False,False,True,False,True,False,False


In [20]:
category_columns = list(filter(lambda x: not isinstance(new_df.iloc[0, x], np.float64), range(new_df.shape[1])))
print(category_columns)

[0, 1, 2, 3, 4, 5, 10, 11, 12, 13, 14, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [21]:
train = pd.read_csv('train_answer.csv')
train.head()

,id,passed
0,5038,0
1,5039,0
2,5040,0
3,5041,1
4,5043,0


In [22]:
X = new_df.loc[train['id'].to_numpy()]
y = train.set_index('id')['passed']
print(sum(y)/len(y))

0.13955756363726143


**Обучение модели**

In [23]:
X_t, X_v, y_t, y_v = train_test_split(X, y, random_state=0, shuffle=True)

In [24]:
params = {'loss_function':'Logloss',
          'eval_metric':'AUC',
          'cat_features': category_columns,
          'verbose': 200,
          'random_seed': 0,
          'iterations': 15000
         }
model = CatBoostClassifier(**params)
model.fit(X_t, y_t,
          eval_set=(X_v, y_v),
          use_best_model=True,
          plot=True
         )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.026906
0:	test: 0.7452852	best: 0.7452852 (0)	total: 88.8ms	remaining: 22m 11s
200:	test: 0.9065791	best: 0.9065791 (200)	total: 4.54s	remaining: 5m 33s
400:	test: 0.9112195	best: 0.9112195 (400)	total: 9.17s	remaining: 5m 33s
600:	test: 0.9133149	best: 0.9133149 (600)	total: 14s	remaining: 5m 35s
800:	test: 0.9145868	best: 0.9145868 (800)	total: 18.9s	remaining: 5m 35s
1000:	test: 0.9154996	best: 0.9154996 (1000)	total: 23.9s	remaining: 5m 34s
1200:	test: 0.9161666	best: 0.9161666 (1200)	total: 28.9s	remaining: 5m 31s
1400:	test: 0.9168187	best: 0.9168187 (1400)	total: 33.9s	remaining: 5m 29s
1600:	test: 0.9172481	best: 0.9172543 (1586)	total: 39s	remaining: 5m 26s
1800:	test: 0.9175842	best: 0.9175967 (1795)	total: 44s	remaining: 5m 22s
2000:	test: 0.9178916	best: 0.9178916 (2000)	total: 49s	remaining: 5m 18s
2200:	test: 0.9182656	best: 0.9182656 (2200)	total: 54.1s	remaining: 5m 14s
2400:	test: 0.9185706	best: 0.9185841 (2396)	total: 59.3s	remaining: 5m 11s
26

In [26]:
test = pd.read_csv('sample_submission.csv')
test.head()

,id,passed
0,46050,0
1,93905,0
2,79028,0
3,101050,0
4,83386,0


In [27]:
X_test = new_df.loc[test['id'].to_numpy()]

In [28]:
pred = model.predict_proba(X_test)

In [29]:
pd.Series(pred[:,1])

0       0.007283
1       0.002345
2       0.029305
3       0.331980
4       0.016880
          ...   
3995    0.054328
3996    0.700483
3997    0.018627
3998    0.042728
3999    0.024116
Length: 4000, dtype: float64

In [30]:
test = test.set_index('id')
test['passed'] = pred[:,1]
test.head()

,passed
id,
46050,0.007283
93905,0.002345
79028,0.029305
101050,0.331980
83386,0.016880


In [31]:
test.to_csv('my_submission.csv')

**Топ-50 наиболее важных признаков**

In [32]:
A = model.get_feature_importance()
B = A.argsort()[::-1]
feature_importances = pd.DataFrame({'feature': new_df.columns[B], 'importance': A[B]})

In [33]:
feature_importances.head(50)

,feature,importance
0,added_by,19.159483
1,developer,12.894421
2,views_num,11.973629
3,responsible,10.865838
4,impact_knowledge,5.929504
5,act_type,4.389912
6,month,4.365790
7,mineco_solution,4.036151
8,day,3.940443
9,comments_num,1.340724
